# Tim Hortons Simulation Code

Note: This does not need to be an ipynb file, but I figure this might be simplest. 

In [231]:
############
# PACKAGES #
############

import numpy as np
import heapq # Event tree package


In [232]:
##############
# PARAMETERS #
##############

# FRONTEND PARAMETERS
NUM_CASHIERS = 2
MEAN_DRIVE_THRU_INTERARRIVAL = 4
MEAN_WALK_IN_INTERARRIVAL = 2
MEAN_MOBILE_INTERARRIVAL = 10

MEAN_WINDOW1_SERVICE = 0.5
MEAN_WINDOW2_SERVICE = 0.5
MEAN_CASHIER_SERVICE = 1

# FOOD PARAMETERS
MEAN_PANINI_TIME = 2
MEAN_HASHBROWN_TIME = 0.2
MEAN_SANDWICH_TIME = 1

MEAN_COFFEE_TIME = 0.5
MEAN_ESPRESSO_TIME = 1
MEAN_DONUT_TIME = 0.2

# KITCHEN PARAMETERS
NUM_COOKS = 5
NUM_BARISTAS = 5

NUM_PANINI = 2
NUM_SANDWICH = 2
NUM_HASHBROWN_STATIONS = 1

NUM_COFFEE_MAKER = 1
NUM_COFFEE_URN = 1
NUM_DONUT_STATIONS = 1

In [233]:
###########
# CLASSES #
###########

# NOTE - We can change classes as needed

class Order:
    # Intialize an order object
    def __init__(self, order_id, type, num_customers, num_items, creation_time, expected_time):
        self.order_id = order_id           # Unique identifier attached to child food items as well
        self.num_customers = num_customers # Number of customers for seating, irrelevant to drive-thru
        self.num_items = num_items         # Number of customers for checking condition of order completion
        self.items_completed = 0           # Number of items completed, must equal num_items to complete order
        self.creation_time = creation_time # Time the order is first instantiated (once order enters customer service queue)
        self.expected_time = expected_time # (Only relevant to mobile orders)

        # Walk-in, pick-up, or drive-thru
        if (type.lower() == "walk-in") or (type.lower() == "drive-thru") or (type.lower() == "mobile"):
            self.order_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid order type string ", type.lower())


class Food:
    # Initialize food object, each order has many corresponding food items
    def __init__(self, food_id, order_id, type, creation_time, expected_time):
        self.food_id = food_id             # Id for food item
        self.order_id = order_id           # Corresponding order
        self.food_type = type              # Type of food item (i.e. coffee, sandwich, etc.)
        self.creation_time = creation_time # Time the item is first instantiated (once order enters customer service queue)
        self.expected_time = expected_time # (Only relevant to mobile orders)

        # Type of food item (coffee, espresso, donut, panini, hashbrown, sandwich, etc.)
        if (type.lower() == "coffee") or (type.lower() == "espresso") or (type.lower() == "donut") or (type.lower() == "panini") or (type.lower() == "hashbrown") or (type.lower() == "sandwich"):
            self.food_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid food type string ", type.lower())

        if type.lower() == "coffee":
            self.mean_service_time = MEAN_COFFEE_TIME
        elif type.lower() == "espresso":
            self.mean_service_time = MEAN_ESPRESSO_TIME
        elif type.lower() == "donut":
            self.mean_service_time = MEAN_DONUT_TIME
        elif type.lower() == "panini":
            self.mean_service_time = MEAN_PANINI_TIME
        elif type.lower() == "sandwich":
            self.mean_service_time = MEAN_SANDWICH_TIME
        elif type.lower() == "hashbrown":
            self.mean_service_time = MEAN_HASHBROWN_TIME
    

class Staff:
    # Create a staff worker
    def __init__(self, type):
        self.staff_idle = True   # Bool for if the staff is working or not

        # Worker type (i.e. barista, cashier, kitchen, drive-thru window, etc.)
        if (type.lower() == "barista") or (type.lower() == "cashier") or (type.lower() == "kitchen") or (type.lower() == "cafe"):
            self.staff_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid food type string ", type.lower())
    
    

class Equipment:
    # Create a piece of equipment
    def __init__(self, type, num_slots):
        self.eq_type = type        # The equipment type
        self.num_slots = num_slots # The total number of people who can work at a station/equipment
        self.used_slots = 0        # The number of equipment slots currently in use

        # Equipment type (i.e. panini-press, sandwich-station, coffee-maker, coffee-urn, etc.)
        if (type.lower() == "panini-press") or (type.lower() == "sandwich-station") or (type.lower() == "coffee-maker") or (type.lower() == "coffee-urn") or (type.lower() == "cash-register") or (type.lower() == "hashbrown-station") or (type.lower() == "donut-station"):
            self.eq_type = type.lower()
        else:
            # Prevent bugs
            raise NameError("Invalid food type string ", type.lower())



class Event:
    def __init__(self, time, event_function, obj):
        self.time = time               # Time that the event must occur
        self.func = event_function     # Function which triggers upon event
        self.obj = obj                 # Object assocaited with event (i.e. Food, Order, etc.)

    # Less than function used for heapq
    # The choice is arbitrary to us, so just return True
    def __lt__(self, other):
        return True

In [234]:
###########################
# EVENT TREE INSTRUCTIONS #
###########################
##
# NOTE - The heapq package handles all the logic, and sorts by minimul value
##

# To add an event object to the event tree in O(log(n)), do the following:
"""  
time = some_event_time
event_data = Event(some_stuff)
heapq.heappush(event_tree, (time, event_id, event_data))
"""

# Then to pop the event off of the event tree in O(log(n)), do the following:
"""
time, event = heapq.heappop(event_queue)
"""

# Or to peek at the next event in O(1), do the following:
"""
time, event = event_queue[0]
"""


'\ntime, event = event_queue[0]\n'

In [235]:
class TimHortons:

    ##################
    ##################
    #                #
    # INITIALIZAITON #
    #                #
    ##################
    ##################
    
    def __init__(self, num_customers_required):
        ############
        # FRONTEND #
        ############
        self.cashiers = [Staff("cashier") for i in range(NUM_CASHIERS)]
        self.drive_thru_window1_status = 0
        self.drive_thru_window2_status = 0

        # current not being used, but can easily implement balking
        self.drive_thru_length = 5
        self.walk_in_length = 5

        self.next_order_id = 0 # indeixng starts at 0
        self.next_food_id = 0 # indexing starts at 0
        
        self.drive_thru_window1_queue = []
        self.drive_thru_window2_queue = []
        self.walk_in_queue = []
        
        self.order_dict = {} # dictionary of orders, the key is the order id

        ###########
        # KITCHEN #
        ###########
        # Staff
        self.cooks = [Staff("kitchen") for i in range(NUM_COOKS)]
        self.baristas = [Staff("cafe") for i in range(NUM_BARISTAS)]

        # Equipment
        self.panini_press = Equipment("panini-press", NUM_PANINI)
        self.sandwich_station = Equipment("sandwich-station", NUM_SANDWICH)
        self.hashbrown_station = Equipment("hashbrown-station", NUM_HASHBROWN_STATIONS)
        self.coffee_maker = Equipment("coffee-maker", NUM_COFFEE_MAKER)
        self.coffee_urn = Equipment("coffee-urn", NUM_COFFEE_URN)
        self.donut_station = Equipment("donut-station", NUM_DONUT_STATIONS)

        # Queues
        self.event_queue = []
        self.kitchen_queue = []
        self.cafe_queue = []
        self.output = [] # TODO - REMOVE THIS!! THIS IS WHERE MY STUFF GOES

        # Ids
        self.next_food_id = 0
        self.next_order_id = 0

        # TODO - REPLACE THIS
        self.interarrival_time = 0.5

        ###########
        # BACKEND #
        ###########
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!
        # !!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!TODO!!!

        ##########
        # SHARED #
        ##########
        # State variables
        self.sim_time = 0
        self.time_last_event = 0.0
        self.num_customers_required = num_customers_required
        self.current_num_customers = 0

        # Stats
        self.total_time_in_kitchen = 0
        self.area_in_kitchen_queue = 0
        self.area_in_cafe_queue = 0

        ##################
        # INITIAL EVENTS #
        ##################
        # Drive-thru event
        initial_drive_thru_arrival = self.sim_time+self.expon(MEAN_DRIVE_THRU_INTERARRIVAL)
        heapq.heappush(self.event_queue, (initial_drive_thru_arrival, Event(initial_drive_thru_arrival, self.arrival, "drive-thru")))
        
        # Walk-in event
        initial_walk_in_arrival = self.sim_time+self.expon(MEAN_WALK_IN_INTERARRIVAL)
        heapq.heappush(self.event_queue, (initial_walk_in_arrival, Event(initial_walk_in_arrival, self.arrival, "walk-in")))

        # Mobile event
        initial_mobile_arrival = self.sim_time+self.expon(MEAN_MOBILE_INTERARRIVAL)
        heapq.heappush(self.event_queue, (initial_mobile_arrival, Event(initial_mobile_arrival, self.arrival, "mobile")))









    ####################
    ####################
    #                  #
    # HELPER FUNCTIONS #
    #                  #
    ####################
    ####################

    def expon(self, mean):
        """Function to generate exponential random variates."""
        return -mean * np.log(np.random.uniform(0, 1))


    def dummy2(self, value):
        return "There"
    
    
    # priority for food and beverage items
    def food_priority(self, food):
        return -food.creation_time


    def remove_from_queue(self, food, queue):
        for i in range(len(queue)):
            if food.food_id == queue[i].food_id:
                del queue[i]
                return
            
        print("Error: food not found in queue")


    # returns food in given array that have available equipment
    def get_possible_food(self, food_array):
        possible_food = []

        for food in food_array:
            if self.is_equipment_available(self.get_equipment(food)):
                possible_food.append(food)

        return possible_food


    def is_equipment_available(self, equip):
        return equip.num_slots > equip.used_slots
    

    def get_equipment(self, food):
        type = food.food_type
        if type == "panini":
            return self.panini_press
        elif type == "sandwich":
            return self.sandwich_station
        elif type == "hashbrown":
            return self.hashbrown_station
        elif type == "coffee":
            return self.coffee_urn
        elif type == "espresso":
            return self.coffee_maker
        elif type == "donut":
            return self.donut_station
        
        
    def is_kitchen_item(self, food):
        type = food.food_type
        if (type == "panini") or (type == "sandwich") or (type == "hashbrown"):
            return True
        else:
            return False
    

    def is_cafe_item(self, food):
        type = food.food_type
        if (type == "coffee") or (type == "espresso") or (type == "donut"):
            return True
        else:
            return False
        

    def is_queue_empty(self, food):
        return (self.is_kitchen_item(food) and len(self.kitchen_queue) == 0) \
            or (self.is_cafe_item(food) and len(self.cafe_queue) == 0)








    ###################
    ###################
    #                 #
    # FRONTEND EVENTS #
    #                 #
    ###################
    ###################

    def generate_order(self, order_type):
        num_drinks = 2 # TODO - CHANGE THIS TO BE A RANDOM NUMBER
        num_food = 2 # TODO - CHANGE THIS TO BE A RANDOM NUMBER
        num_customers = 2 # not sure if this is relevant information for dining.... but will asign a value anyways, TODO - CHANGE THIS TO BE A RANDOM NUMBER
    
        drink = np.random.choice(["coffee", "espresso", "donut"], size=num_drinks) # uniform by default, but probabilities can be assigned
        food = np.random.choice(["panini", "sandwich", "hashbrown"], size=num_food) # uniform by default, but probabilities can be assigned
    
        if order_type == "mobile":
            expected_time = self.sim_time + 20 # current time + 20 minutes, TODO - CHANGE THIS TO BE A RANDOM NUMBER
        else:
            expected_time = np.inf
            
        # CREATING ORDER OBJECT
        order_id = self.next_order_id
        order_info = Order(order_id, order_type, num_customers, len(drink)+len(food), self.sim_time, expected_time)
        self.order_dict[order_id] = order_info

        # CREATING FOOD OBJECT FOR ALL FOOD ITEMS TO BE GIVEN TO KITCHEN
        for item in np.concatenate((drink, food), axis=0):
            food = Food(self.next_food_id, self.next_order_id, item, self.sim_time, expected_time)
            food_event = Event(self.sim_time, self.handle_kitchen_arrival, food)
            heapq.heappush(self.event_queue, (self.sim_time, food_event))
            self.next_food_id += 1 # updating the food index
    
        self.next_order_id += 1 # updating order index

        return order_id
        
    
    def place_order(self, order_location):
        order_type, server_pos = order_location
        order_id = self.generate_order(order_type) # generate an order event because we have been serviced
        self.current_num_customers += 1 # REMOVE THIS, CURRENTLY CAUSES LOOP TO STOP

        if order_type == "walk-in":
            # CHECKING IF ANYONE IS IN THE QUEUE
            if len(self.walk_in_queue) > 0:
                order_time = self.sim_time+self.expon(MEAN_CASHIER_SERVICE)
                order_event = Event(order_time, self.place_order, (order_type, server_pos))
                heapq.heappush(self.event_queue, (order_time, order_event))
                del self.walk_in_queue[0]

            else:
                self.cashiers[server_pos].staff_idle = True
                
        elif order_type == "drive-thru":
            # SCHEDULING THE ARRIVAL AT PAYMENT WINDOW
            payment_time = self.sim_time+self.expon(MEAN_WINDOW2_SERVICE)
            payment_event = Event(payment_time, self.dummy2, order_id) # arrival
            heapq.heappush(self.event_queue, (payment_time, payment_event))

            # CHECKING IF ANYONE IS IN THE QUEUE
            if len(self.drive_thru_window1_queue) > 0:
                order_time = self.sim_time+self.expon(MEAN_WINDOW1_SERVICE)
                order_event = Event(order_time, self.place_order, (order_type, server_pos))
                heapq.heappush(self.event_queue, (order_time, order_event))
                del self.drive_thru_window1_queue[0]

            else:
                self.drive_thru_window1_status = 0


    def arrival(self, arrival_type):
        if arrival_type.lower() == "walk-in":
            # SCHEDULE NEXT ARRIVAL
            walk_in_arrival_time = self.sim_time+self.expon(MEAN_WALK_IN_INTERARRIVAL)
            heapq.heappush(self.event_queue, (walk_in_arrival_time, Event(walk_in_arrival_time, self.arrival, "walk-in")))

            # CHECKING IF CASHIERS ARE BUSY
            first_free_cashier = -1
            i = 0
            for cashier in self.cashiers:
                if cashier.staff_idle:
                    first_free_cashier = i
                i += 1
                
            if first_free_cashier < 0:
                self.walk_in_queue.append(self.sim_time) # all cashiers are busy, so add customer to queue
            else:
                self.cashiers[first_free_cashier].staff_idle = False
                # CREATING A PLACE ORDER EVENT
                order_time = self.sim_time+self.expon(MEAN_CASHIER_SERVICE)
                order_event = Event(order_time, self.place_order, (arrival_type, first_free_cashier))
                heapq.heappush(self.event_queue, (order_time, order_event))
                
        elif arrival_type.lower() == "drive-thru":
            # SCHEDULE NEXT ARRIVAL 
            drive_thru_arrival_time = self.sim_time+self.expon(MEAN_DRIVE_THRU_INTERARRIVAL)
            heapq.heappush(self.event_queue, (drive_thru_arrival_time, Event(drive_thru_arrival_time, self.arrival, "drive-thru")))

            # CHECKING IF DRIVE-THRU WINDOW 1 IS BUSY
            if self.drive_thru_window1_status == 1:
                self.drive_thru_window1_queue.append(self.sim_time) # add to queue if busy
            else:
                self.drive_thru_window1_status = 1 # make window 1 server busy
                # CREATING A PLACE ORDER EVENT 
                order_time = self.sim_time+self.expon(MEAN_WINDOW1_SERVICE)
                order_event = Event(order_time, self.place_order, (arrival_type, 0)) # server position is 0 because there is only one server
                heapq.heappush(self.event_queue, (order_time, order_event))
                
        elif arrival_type.lower() == "mobile":
            # SCHEDULE NEXT ARRIVAL
            mobile_arrival_time = self.sim_time+self.expon(MEAN_MOBILE_INTERARRIVAL)
            heapq.heappush(self.event_queue, (mobile_arrival_time, Event(mobile_arrival_time, self.arrival, "mobile")))
            
            # CREATING A PLACE ORDER EVENT
            order_event = Event(self.sim_time, self.place_order, (arrival_type,0)) # assuming there is no service time for mobile order
            heapq.heappush(self.event_queue, (self.sim_time, order_event))
            
        else:
            raise NameError("Invalid order type string ", arrival_type.lower())









##################
##################
#                #
# KITCHEN EVENTS #
#                #
##################
##################

    def handle_kitchen_arrival(self, food):
        # If corresponding equipment is unavailable
        equip = self.get_equipment(food)
        if not self.is_equipment_available(equip):
            # Put onto correct queue
            if self.is_kitchen_item(food):
                self.kitchen_queue.append(food)
            else:
                self.cafe_queue.append(food)
            return

        # Select worker set
        if self.is_kitchen_item(food):
            workers = self.cooks
        else:
            workers = self.baristas

        # If any workers are idle
        for worker in workers:
            if worker.staff_idle:
                # assign cook and equipment slot
                worker.staff_idle = False
                equip.used_slots += 1

                # schedule departure
                time = self.sim_time + self.expon(food.mean_service_time)
                obj = (time, food, worker)
                event = Event(time, self.handle_kitchen_departure, obj)
                heapq.heappush(self.event_queue, (time, event))

                return
        
        # If equipment is available but no cooks, join queue
        self.kitchen_queue.append(food)



    def handle_kitchen_departure(self, obj):
        time, food, worker = obj

        # TODO - REMOVE THIS!! THIS IS WHERE MY STUFF GOES
        self.output.append(food)

        equip = self.get_equipment(food)
        equip.used_slots -= 1

        # No food in queue
        if self.is_queue_empty(food):
            worker.staff_idle = True
        else:
            # get next food that has an available equipment
            if self.is_kitchen_item(food):
                possible_food = self.get_possible_food(self.kitchen_queue)
            else:
                possible_food = self.get_possible_food(self.cafe_queue)

            if possible_food == []:
                worker.staff_idle = True
                return

            # Get food item with max priority
            # TODO - Implement advanced priority system
            next_food = max(possible_food, key=self.food_priority)
            
            # remove food from kitchen_queue
            if self.is_kitchen_item(food):
                self.remove_from_queue(next_food, self.kitchen_queue)
            else:
                self.remove_from_queue(next_food, self.cafe_queue)

            # worker stays busy
            next_equip = self.get_equipment(next_food)
            next_equip.used_slots += 1

            # schedule departure
            time = self.sim_time + self.expon(next_food.mean_service_time)
            obj = (time, next_food, worker)
            event = Event(time, self.handle_kitchen_departure, obj)
            heapq.heappush(self.event_queue, (time, event))






        
    ##################
    ##################
    #                #
    # BACKEND EVENTS #
    #                #
    ##################
    ##################



        
    ##################
    ##################
    #                #
    # MAIN FUNCTIONS #
    #                #
    ##################
    ##################


    ##########
    # REPORT #
    ##########
    def report(self):
        print(f"avg number in kitchen_queue: {self.area_in_kitchen_queue / self.sim_time}")
        print(f"avg number in cafe_queue: {self.area_in_cafe_queue / self.sim_time}")
        print(f"Total sim_time: {self.sim_time}")


    #########
    # STATS #
    #########
    def update_time_avg_stats(self):
        time_since_last_event = self.sim_time - self.time_last_event

        self.area_in_kitchen_queue += len(self.kitchen_queue) * time_since_last_event
        self.area_in_cafe_queue += len(self.cafe_queue) * time_since_last_event

        self.time_last_event = self.sim_time


    ########
    # MAIN #
    ########
    def main(self):
        while (self.current_num_customers < self.num_customers_required):
            self.update_time_avg_stats()

            time, event = heapq.heappop(self.event_queue)
            self.sim_time = time # updating the sim time

            event.func(event.obj)

        self.report()
    
test_obj = TimHortons(1000)
test_obj.main()

avg number in kitchen_queue: 0.7058080682732875
avg number in cafe_queue: 0.8904244811712658
Total sim_time: 1164.6187495193371
